In [ ]:
# SCRIPT TO, BASED ON A HUBSPOT XLSX FILE, EXTRACT THE INDUSTRY OF ANY GIVEN COMPANY (THAT WASNT YET MAPPED) - BASED ON LINKEDIN INFORMATION


In [ ]:
from os.path import dirname, basename, isfile

import ast
import time

from . import actions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from collections import OrderedDict
from selenium import webdriver

In [ ]:
df = pd.read_excel('other_industries.xlsx')  
df_limited = df[['First Name', 'Last Name', 'Company Name', 'Email']]

In [ ]:
driver = webdriver.Chrome()

email = "LINKEDIN EMAIL"
password = "LINKEDIN PWD"
actions.login(driver, email, password)


In [ ]:
companies = []

In [ ]:
from tqdm import tqdm 

for i in tqdm(df_limited['Company Name'].dropna().unique()):
    
    
    linkedin_industry, linkedin_company_name, linkedin_url, linkedin_summary = [None] *4
    
    try:
        driver.get(f"https://www.linkedin.com/search/results/companies/?keywords={i}&origin=SWITCH_SEARCH_VERTICAL&sid=dFD")

        linkedin_industry = driver.find_element(By.CLASS_NAME, 'entity-result__primary-subtitle').text.split(' • ')[0]
        linkedin_company_name = driver.find_element(By.CLASS_NAME, 'entity-result__title-text').text
        linkedin_url = driver.find_element(By.CLASS_NAME, 'entity-result__title-text').find_element(By.TAG_NAME, "a").get_attribute('href')
    except:
        pass
    try:
        linkedin_summary = driver.find_element(By.CLASS_NAME, 'entity-result__summary').text
    except:
        pass
    
    companies.append([i, linkedin_industry, linkedin_company_name, linkedin_url, linkedin_summary])
    time.sleep(2)
        
        

In [ ]:
company_df = pd.DataFrame(companies, columns = ['Company Name', 'linkedin_industry', 'linkedin_company', 'linkedin_link', 'linkedin_summary'])

In [ ]:
import re

def regex_name(row):
    try: 
        c = row['Company Name'].lower().strip()
        n = row['linkedin_company'].lower().strip()
        
        if re.search(c, n):
            return True

        elif re.search(n, c):
            return True
    except:
        pass
    
    return False

In [ ]:
company_df['regex_match'] = company_df.apply(regex_name, axis=1)

In [ ]:
company_df.head()

In [ ]:
company_df.loc[company_df.linkedin_industry.isna()].shape

In [ ]:
df3 = df_limited.merge(company_df, how='outer', on='Company Name')

In [ ]:
personal_domains = ['aol',
 'gmail',
 'gmx',
 'googlemail',
 'hotmail',
 'hotmail.co',
 'icloudlive',
 'live.co',
 'outlook',
 'web',
 'yahoo',
 'yahoo.co',
 'yahoo.com',
 'ymail']

In [ ]:
email = []
for email in df3.loc[(df3.linkedin_industry.isna()) & (df3['Company Name'].isna())].Email.values:

    if not email is np.nan:
        if not any(ele in email for ele in personal_domains):
            pattern = r"@([A-Za-z0-9.-]+)\."
            match = re.findall(pattern, email)
            query_company = match[0].split('.')[0]
    
            linkedin_industry, linkedin_company_name, linkedin_url, linkedin_summary = [None] *4

            try:

                # maybe think about preprocessing of company name. eg Lower case, remove spa etc...
                driver.get(f"https://www.linkedin.com/search/results/companies/?keywords={query_company}&origin=SWITCH_SEARCH_VERTICAL&sid=dFD")

                linkedin_industry = driver.find_element(By.CLASS_NAME, 'entity-result__primary-subtitle').text.split(' • ')[0]
                linkedin_company_name = driver.find_element(By.CLASS_NAME, 'entity-result__title-text').text
                linkedin_url = driver.find_element(By.CLASS_NAME, 'entity-result__title-text').find_element(By.TAG_NAME, "a").get_attribute('href')
            except:
                pass
            try:
                linkedin_summary = driver.find_element(By.CLASS_NAME, 'entity-result__summary').text
            except:
                pass

            email.append([email, linkedin_industry, linkedin_company_name, linkedin_url, linkedin_summary])
            time.sleep(2)
            

In [ ]:
df_email = pd.DataFrame(email, columns = ['Email', 'linkedin_industry', 'linkedin_company', 'linkedin_link', 'linkedin_summary'])


def merge_email(row):

    email = row['Email']
    df3.loc[df3['Email'] == email, 'linkedin_industry'] = row['linkedin_industry']
    df3.loc[df3['Email'] == email, 'linkedin_company'] = row['linkedin_company']
    df3.loc[df3['Email'] == email, 'linkedin_link'] = row['linkedin_link']
    df3.loc[df3['Email'] == email, 'linkedin_summary'] = row['linkedin_summary']
    

_ = df_email.apply(merge_email, axis = 1)

In [ ]:
df3.to_excel("output.xlsx")  